Plant Parenthood

In [11]:
%load_ext sql


The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [2]:
%sql sqlite:///Plant_Parenthood2.db

In [3]:
import pymysql

In [4]:
%%sql
CREATE TABLE CUSTOMERS(
    cid INT,
    name VARCHAR(255) NOT NULL,
    email TEXT);

 * sqlite:///Plant_Parenthood2.db
(sqlite3.OperationalError) table CUSTOMERS already exists
[SQL: CREATE TABLE CUSTOMERS(
    cid INT,
    name VARCHAR(255) NOT NULL,
    email TEXT);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [5]:
%%sql
CREATE TABLE PLANTS(
    pid INTEGER PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    author TEXT);

 * sqlite:///Plant_Parenthood2.db
(sqlite3.OperationalError) table PLANTS already exists
[SQL: CREATE TABLE PLANTS(
    pid INTEGER PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    author TEXT);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [6]:
%%sql
CREATE TABLE ORDERS(
    oid INT PRIMARY KEY, 
    orderDate VARCHAR(255) NOT NULL,
    shippingAddress TEXT);

 * sqlite:///Plant_Parenthood2.db
(sqlite3.OperationalError) table ORDERS already exists
[SQL: CREATE TABLE ORDERS(
    oid INT PRIMARY KEY, 
    orderDate VARCHAR(255) NOT NULL,
    shippingAddress TEXT);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [7]:
%%sql
CREATE TABLE CONTAINS(
    oid INTEGER,
    pid INTEGER,
    PRIMARY KEY (oid, pid),
    FOREIGN KEY (oid) REFERENCES ORDERS(oid) ON DELETE CASCADE,
    FOREIGN KEY (pid) REFERENCES BOOKS(pid) ON DELETE CASCADE
);

 * sqlite:///Plant_Parenthood2.db
(sqlite3.OperationalError) table CONTAINS already exists
[SQL: CREATE TABLE CONTAINS(
    oid INTEGER,
    pid INTEGER,
    PRIMARY KEY (oid, pid),
    FOREIGN KEY (oid) REFERENCES ORDERS(oid) ON DELETE CASCADE,
    FOREIGN KEY (pid) REFERENCES BOOKS(pid) ON DELETE CASCADE
);]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [8]:
%%sql
INSERT INTO CUSTOMERS (cid, name, email)
VALUES (9, 'Joe mama', 'joe_mama@gmail.com');

 * sqlite:///Plant_Parenthood2.db
1 rows affected.


[]

In [7]:
%%sql
CREATE TABLE user_preferences (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT UNIQUE NOT NULL,
    theme TEXT NOT NULL DEFAULT 'light'
);

 * sqlite:///Plant_Parenthood2.db
Done.


[]

In [9]:
%%sql
select * from customers

 * sqlite:///Plant_Parenthood2.db
Done.


cid,name,email
9,Joe mama,joe_mama@gmail.com


In [ ]:
%%sql
INSERT INTO ORDERS (oid, orderDate, shippingAddress)
VALUES (5, '2024-08-09', '100 Chapel Hill Dr, Chapel Hill, NC');

In [12]:
%%sql 
select * from user_preferences

 * sqlite:///Plant_Parenthood2.db
Done.


id,username,theme


In [10]:
import sqlite3

# Connect to the database (or create it if it doesn't exist)
conn = sqlite3.connect('Plant_Parenthood.db')
cursor = conn.cursor()

# Fetch the list of tables in the database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [table[0] for table in cursor.fetchall()]

# Check if tables exist in the database
if not tables:
    print("No tables found in the database. Please create a table before proceeding.")
    conn.close()
    exit()

def display_menu():
    # Display available tables and let the user choose one
    print("\nAvailable tables:")
    for i, table in enumerate(tables, start=1):
        print(f"{i}. {table}")

    try:
        table_choice = int(input("Select the table you want to alter (by number): ")) - 1
        if table_choice < 0 or table_choice >= len(tables):
            raise ValueError("Invalid table choice.")
        return tables[table_choice]
    except ValueError:
        print("Invalid choice. Exiting.")
        conn.close()
        exit()

def perform_action(selected_table):
    # Menu for selecting an action
    print("\nChoose an action:")
    print("1. Add a new record")
    print("2. Delete a record")
    print("3. View all records")
    print("4. Update a record")
    choice = input("Enter your choice: ")

    if choice == "1":
        # Add a new record
        print("\nAdding a new record:")
        columns_query = f"PRAGMA table_info({selected_table});"
        cursor.execute(columns_query)
        columns = [col[1] for col in cursor.fetchall()]
        
        values = []
        for column in columns:
            value = input(f"Enter value for {column}: ")
            values.append(value)

        placeholders = ", ".join(["?"] * len(columns))
        query = f"INSERT INTO {selected_table} ({', '.join(columns)}) VALUES ({placeholders})"
        
        try:
            cursor.execute(query, values)
            conn.commit()
            print("New record added successfully!")
        except sqlite3.IntegrityError as e:
            print(f"Error: {e}")

    elif choice == "2":
        # Delete a record
        delete_column = input(f"Enter the column name to filter by for deletion: ")
        delete_value = input(f"Enter the value for {delete_column} to delete: ")
        
        try:
            cursor.execute(f"DELETE FROM {selected_table} WHERE {delete_column} = ?", (delete_value,))
            conn.commit()
            print("Record deleted successfully!")
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")

    elif choice == "3":
        # View all records
        print("\nViewing all records:")
        try:
            cursor.execute(f"SELECT * FROM {selected_table}")
            rows = cursor.fetchall()
            for row in rows:
                print(row)
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")

    elif choice == "4":
        # Update a record
        print("\nUpdating a record:")
        update_column = input("Enter the column name you want to update (e.g., Email): ")
        new_value = input(f"Enter the new value for {update_column}: ")
        filter_column = input("Enter the column name to filter by (e.g., CustomerID): ")
        filter_value = input(f"Enter the value of {filter_column} to identify the record: ")
        
        try:
            cursor.execute(
                f"UPDATE {selected_table} SET {update_column} = ? WHERE {filter_column} = ?",
                (new_value, filter_value)
            )
            conn.commit()
            print("Record updated successfully!")
        except sqlite3.OperationalError as e:
            print(f"Error: {e}")

    else:
        print("Invalid choice. Returning to main menu.")

while True:
    selected_table = display_menu()
    perform_action(selected_table)

    # Ask the user if they want to perform another action
    continue_choice = input("\nDo you want to perform another action? (yes/no): ").strip().lower()
    if continue_choice != "yes":
        print("Exiting program. Goodbye!")
        break

# Close the connection
conn.close()



Available tables:
1. PLANTS
2. ORDERS
3. CONTAINS
4. CUSTOMERS

Choose an action:
1. Add a new record
2. Delete a record
3. View all records
4. Update a record

Adding a new record:
New record added successfully!
Exiting program. Goodbye!


In [6]:
%%sql
select * From Customers


 * sqlite:///Plant_Parenthood.db
Done.


cid,name,email
,1,2


In [7]:
%%sql
SELECT name FROM sqlite_master WHERE type='table';

 * sqlite:///Plant_Parenthood.db
Done.


name
PLANTS
ORDERS
CONTAINS
CUSTOMERS
